- DataFrame을 다루는 기능
    - Row 타입의 레코드와 각 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼으로 구성
    - 스키마
    - 파티셔닝
    - 파티셔닝 스키마 : 파티션을 배치하는 방법을 정의

- DataFrame 생성

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.2:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1681772094003)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5450c6e8


In [4]:
val df = spark.read.format("json").load("sample_data/flight-data/json/2015-summary.json")

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


- 스키마 살펴보기

In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



# 5.1 스키마

In [7]:
spark.read.format("json").load("sample_data/flight-data/json/2015-summary.json").schema

res2: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true))


- 스키마는 여러 개의 StructField 타입으로 구성된 StructType 객체
- StructField는 이름, 데이터 타입, 컬럼이 값이 없거나 null일 수 있는지 지정하는 불리언 값을 가짐

- DataFrame에 스키마를 만들고 적용

In [9]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata

val myManualSchema = StructType(Array(
    StructField("DEST_COUNTRY_NAME", StringType, true),
    StructField("ORIGIN_COUNTRY_NAME", StringType, true),
    StructField("count", LongType, false,
                Metadata.fromJson("{\"hello\":\"world\"}"))
    ))

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata
myManualSchema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,false))


## 5.2 컬럼과 표현식
- 표현식으로 DataFrame의 컬럼을 선택, 조작, 제거할 수 있음
- DataFrame을 통하지 않으면 외부에서 컬럼에 접근할 수 없음
- 컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션을 사용해야 함

## 5.2.1 컬럼
- col, column함수를 이용해 컬럼을 생성 및 참조
- 컬럼명을 인수로 받음

In [10]:
import org.apache.spark.sql.functions.{col, column}

col("someColumnName")
column("someColumnName")

import org.apache.spark.sql.functions.{col, column}
res3: org.apache.spark.sql.Column = someColumnName


- 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지는 미확인 상태로 남음
- 컬럼이 DataFrame에 있을지 없을지는 알 수 없음
- 분석기가 동작하는 단계에서 컬럼과 테이블을 분석함

### 명시적 컬럼 참조
- col 메서드로 참조
- 조인 시 유용

In [11]:
df.col("count")

res4: org.apache.spark.sql.Column = count


## 5.2.2 표현식
- DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합
- 여러 컬럼을 입력으로 받아 식별하고, '단일 값'을 만들기 위해 다양한 표현식을 각 레코드에 적용하는 함수
- expr 함수로 가장 간단히 사용 가능
- 예) expr("someCol")은 col("someCol") 구문과 동일하게 동작

### 표현식으로 컬럼 표현
- 컬럼은 단지 표현식일 뿐
- 컬럼과 컬럼의 트랜스포메이션은 파싱된 표현식과 동일한 논리적 실행 계획으로 컴파일됨

(((col("someCol")+5)*200)-6) < col("otherCol")

In [12]:
import org.apache.spark.sql.functions.expr

expr("(((someCol + 5) * 200) - 6) < otherCol")

import org.apache.spark.sql.functions.expr
res5: org.apache.spark.sql.Column = ((((someCol + 5) * 200) - 6) < otherCol)


# 5.3 레코드와 로우
- 값을 생성하기 위한 컬럼 표현식으로 Row 객체를 다룸

In [15]:
// DataFrame의 first 메서드로 로우 확인
df.first()

res7: org.apache.spark.sql.Row = [United States,Romania,15]


## 5.3.1 로우 생성하기
- Row 객체는 스키마 정보를 가지고 있지 않음(DataFrame만 스키마를 가짐)
- DataFrame의 스키마와 같은 순서로 값을 명시해야 함

In [16]:
import org.apache.spark.sql.Row

val myRow = Row("Hello", null, 1, false)

import org.apache.spark.sql.Row
myRow: org.apache.spark.sql.Row = [Hello,null,1,false]


In [17]:
//로우의 데이터에 접근
myRow(0)

res8: Any = Hello


# 5.4 DataFrame의 트랜스포메이션
- 로우나 컬럼 추가
- 로우나 컬럼 제거
- 로우를 컬럼으로 변환하거나, 그 반대로 변환
- 컬럼값을 기준으로 로우 순서 변경

## 5.4.1 DataFrame 생성하기

In [21]:
//DataFrame을 생성
val df = spark.read.format("json").load("sample_data/flight-data/json/2015-summary.json")

//생성한 DataFrame을 SQL 쿼리를 실행하고 SQL의 기본 트랜스포메이션을 확인하기 위해 임시 뷰로 등록
df.createOrReplaceTempView("dfTable")

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


## 5.4.2 select와 selectExpr

In [22]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [23]:
//selectExpr 메서드
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



- selectExpr 메서드는 새로운 DataFrame을 생성하는 복잡한 표현식을 간단하게 만드는 도구

In [25]:
//column 추가 예제
df.selectExpr(
    "*", //모든 원본 컬럼 포함
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



## 5.4.3 스파크 데이터 타입으로 변환하기
- 새로운 값이 아닌 명시적인 값을 스파크에 전달해야 할 때
- 추후에 비교에 사용할 무언가가 될 수 있음
- 리터럴 사용

In [26]:
import org.apache.spark.sql.functions.lit

df.select(expr("*"), lit(1).as("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



import org.apache.spark.sql.functions.lit


## 5.4.4 컬럼 추가하기
- withColumn 메서드

In [27]:
df.withColumn("numberOne", lit(2)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        2|
|    United States|            Croatia|    1|        2|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [28]:
//출발지와 도착지가 같은지를 불리언 타입으로 표현
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



## 5.4.5 컬럼명 변경하기
- withColumnRenamed

In [29]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

res20: Array[String] = Array(dest, ORIGIN_COUNTRY_NAME, count)


## 5.4.8 컬럼 제거하기
- drop

In [33]:
df.drop("ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|    United States|   15|
|    United States|    1|
+-----------------+-----+
only showing top 2 rows



In [32]:
df.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



## 5.4.9 컬럼의 데이터 타입 변경하기
- cast

In [34]:
df.withColumn("count2", col("count").cast("string"))

res25: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 2 more fields]


## 5.4.10 로우 필터링하기
- filter

In [35]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [36]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



## 5.4.11 고유한 로우 얻기
- distinct

In [37]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

res28: Long = 256


## 5.4.12 무작위 샘플 만들기
- sample

In [39]:
val seed = 5
val withReplacement = false //복원추출 여부
val fraction = 0.5 //표본 데이터 추출 비율
df.sample(withReplacement, fraction, seed).count()

seed: Int = 5
withReplacement: Boolean = false
fraction: Double = 0.5
res30: Long = 138


## 5.4.13 임의 분할하기
- 머신러닝 알고리즘에서 사용할 학습셋, 검증셋 그리고 테스트셋을 만들 때 주로 사용

In [40]:
val dataFrames = df.randomSplit(Array(0.25, 0.75), seed)
dataFrames(0).count() > dataFrames(1).count()

dataFrames: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field], [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field])
res31: Boolean = false


## 5.4.14 로우 합치기와 추가하기


In [41]:
import org.apache.spark.sql.Row

val schema = df.schema

val newRows = Seq(
    Row("New Country", "Other Country", 5L),
    Row("New Country 2", "Other Country 2", 1L)
    )

val parallelizedRows = spark.sparkContext.parallelize(newRows)

val newDF = spark.createDataFrame(parallelizedRows, schema)

df.union(newDF).where("count = 1").where($"ORIGIN_COUNTRY_NAME" =!= "United States")
.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 2|    1|
+-----------------+-------------------+-----+



import org.apache.spark.sql.Row
schema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true))
newRows: Seq[org.apache.spark.sql.Row] = List([New Country,Other Country,5], [New Country 2,Other Country 2,1])
parallelizedRows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[108] at parallelize at <console>:39
newDF: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


## 5.4.15 로우 정렬하기
- sort, orderBy

In [43]:
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

## 5.4.16 로우 수 제한하기

In [44]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



## 5.4.17 repartition과 coalesce

In [45]:
df.rdd.getNumPartitions

res36: Int = 1


In [46]:
df.repartition(5)

res37: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [48]:
df.repartition(col("DEST_COUNTRY_NAME"))

res39: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [49]:
//목적지를 기준으로 셔플을 수행해 5개의 파티션으로 나누고, 전체 데이터를 셔플 없이 병합하는 예제
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

res40: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


## 5.4.18 드라이버로 로우 데이터 수집하기
 - Executor 에 할당된 RDD 를 모두 Driver Node 로 취합하는 Action 

In [50]:
val collectDF = df.limit(10)
collectDF.take(5)
collectDF.show()
collectDF.show(5, false)
collectDF.collect()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India         

collectDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
res41: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344], [Egypt,United States,15], [United States,India,62], [United States,Singapore,1], [United States,Grenada,62], [Costa Rica,United States,588], [Senegal,United States,40], [Moldova,United States,1])
